Google Colaboratoryでivedoorニュース9カテゴリを分類する自然言語処理の実装について解説します。

※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

本実装の流れは、

1. livedoorニュースをダウンロードして、tsvファイルに変換
2. tsvファイルをPyTorchのtorchtextのDataLoaderに変換
3. クラス分類用のモデルを用意する
4. パラメータの設定
5. 学習の実施
6. テストデータでの性能を確認

1. livedoorニュースをダウンロードして、tsvファイルに変換
ここでは、livedoorニュースをダウンロードし、

本文[tab]クラスラベル
本文[tab]クラスラベル
本文[tab]クラスラベル

の構成のtsvファイルへと変換していきます。

まず、LiveDoorニュースをダウンロードします（乱数のシード固定は掲載を省略しています）。

### Janomeのインストール方法

コンソールにて、

- source activate pytorch_p36
- pip install janome

In [1]:
!pip install janome

     |████████████████████████████████| 19.7 MB 7.3 MB/s 


In [2]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2021-11-20 15:28:20--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  5.88MB/s    in 1.4s    

2021-11-20 15:28:22 (5.88 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



ダウンロードした圧縮ファイルを解凍しカテゴリーの数と内容を確認します。

In [3]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)

['smax', 'livedoor-homme', 'movie-enter', 'CHANGES.txt', 'sports-watch', 'README.txt', 'topic-news', 'peachy', 'kaden-channel', 'dokujo-tsushin', 'it-life-hack']
カテゴリー数: 9
['smax', 'livedoor-homme', 'movie-enter', 'sports-watch', 'topic-news', 'peachy', 'kaden-channel', 'dokujo-tsushin', 'it-life-hack']


カテゴリーではない、ファイルなどもあるので、それを無視します。

ひとつ、ファイルの中身を確認してみましょう。

In [4]:
# ファイルの中身を確認してみる
file_name = "./data/livedoor/text/movie-enter/movie-enter-6255260.txt"

with open(file_name) as text_file:
    text = text_file.readlines()
    print("0：", text[0])  # URL情報
    print("1：", text[1])  # タイムスタンプ
    print("2：", text[2])  # タイトル
    print("3：", text[3])  # 本文

    # 今回は4要素目には本文は伸びていないが、4要素目以降に本文がある場合もある

0： http://news.livedoor.com/article/detail/6255260/

1： 2012-02-07T09:00:00+0900

2： 新しいヴァンパイアが誕生！　ジョニデ主演『ダーク・シャドウ』の公開日が決定

3： 　こんなヴァンパイアは見たことがない！　ジョニー・デップとティム・バートン監督がタッグを組んだ映画『ダーク・シャドウズ（原題）』の邦題が『ダーク・シャドウ』に決定。日本公開日が5月19日に決まった。さらに、ジョニー・デップ演じるヴァンパイアの写真が公開された。



今回は、4要素目に本文が入っていませんが、4要素目以降に本文が入っている場合もあります。

この各ファイルから、タイトルは除いて、本文だけを抽出したtsvファイルを作成したいです。

タイトルの除くのは、タイトルは文章内容の要約であり、さすがにクラス分類のための情報量が多すぎるからです。

本文を取得する前処理関数を定義します。ここでは改行や全角スペースも削除しています。

In [5]:
# 本文を取得する前処理関数を定義


def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text

この定義した前処理関数を利用して、全ファイルを変換します。
livedoorニュースの9カテゴリについて、各カテゴリーごとに処理を実施します。

In [6]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)

リストをpandasのDataFrameに変換します。サイズを確認すると、7,376の文章があることが確認できます。

In [7]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)

df.head()

(7376, 2)


,text,label
0,Optimus cafeが21日からオープン！LGエレクトロニクス・ジャパンは19日、同社の...,smax
1,本日発売L-06D JOJOをベンチマーク！NTTドコモから本日30日（木）に発売開始した「...,smax
2,待望の…待望のUSB経由のネットワーク接続が可能に！！Android情報サイト「AppCom...,smax
3,究極の全部入りスマートフォン！既報のとおり、NTTドコモは16日、携帯電話の2012年夏モデ...,smax
4,Optimus it L-05Dを写真と動画で紹介！既報通り、NTTドコモは16日、今夏に発...,smax


続いて、カテゴリー名を数値に変換する辞書を作成します。
そして、その辞書で数値に置き換えたDataFrameを用意します。

In [8]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

print(dic_id2cat)
print(dic_cat2id)

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)
df.head()

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]
df.head()

{0: 'smax', 1: 'livedoor-homme', 2: 'movie-enter', 3: 'sports-watch', 4: 'topic-news', 5: 'peachy', 6: 'kaden-channel', 7: 'dokujo-tsushin', 8: 'it-life-hack'}
{'smax': 0, 'livedoor-homme': 1, 'movie-enter': 2, 'sports-watch': 3, 'topic-news': 4, 'peachy': 5, 'kaden-channel': 6, 'dokujo-tsushin': 7, 'it-life-hack': 8}


,text,label_index
0,Optimus cafeが21日からオープン！LGエレクトロニクス・ジャパンは19日、同社の...,0
1,本日発売L-06D JOJOをベンチマーク！NTTドコモから本日30日（木）に発売開始した「...,0
2,待望の…待望のUSB経由のネットワーク接続が可能に！！Android情報サイト「AppCom...,0
3,究極の全部入りスマートフォン！既報のとおり、NTTドコモは16日、携帯電話の2012年夏モデ...,0
4,Optimus it L-05Dを写真と動画で紹介！既報通り、NTTドコモは16日、今夏に発...,0


データがシャッフルされておらず、カテゴリーごとに固まっているので、シャッフルします。

In [9]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
df.head()

,text,label_index
0,アラサー女子には「結婚」「出産」「転職」など、あらゆる場面で大きな選択がつきもの。あなたは、...,5
1,デジカメを買おうとしている人に、パナソニックから人気のミラーレス一眼「LUMIX DMC-G...,6
2,世界陸上の開幕が8月27日に迫っている。日本勢の躍進、メダル獲得に期待がかかるが、W杯優勝で...,3
3,著作権保護に関する世界初の多国間条約、ベルヌ条約が締結されたスイス。そのスイス政府の調査によ...,6
4,第22 回小説すばる新人賞を受賞した朝井リョウさんのベストセラー小説がついに映画化！ 突然部...,5


シャッフルされたデータの前2割をテストデータ、残りの8割は訓練&検証データとします。

結果、テストデータが1,475件、訓練&検証データが5,901件となります。

これをtest.tsv、train_eval.tsvとして、それぞれ保存します。

In [10]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df[len_0_2:].to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(df[len_0_2:].shape)

(1475, 2)
(5901, 2)


以上でlivedoorニュースのデータをtsvファイルに変換することができました。

なお、tsvファイルをGoogle Colaboratoryからダウンロードしたい場合は以下のfile.downloadのコメントを外して実行します。

In [11]:
# tsvファイルをダウンロードしたい場合
from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")

2. tsvファイルをPyTorchのtorchtextのDataLoaderに変換

続いて、作成したtsvファイルを、PyTorchで扱えるDataLoaderに変換します。
torchtextを使用します。
形態素解析のMeCabをインストールします。

In [12]:
# MeCabとtransformersの用意
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install transformers==2.9.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

In [13]:
from janome.tokenizer import Tokenizer

j_t = Tokenizer()

text = '機械学習が好きです。'

for token in j_t.tokenize(text):
    print(token)


機械	名詞,一般,*,*,*,*,機械,キカイ,キカイ
学習	名詞,サ変接続,*,*,*,*,学習,ガクシュウ,ガクシュー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
好き	名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。


In [14]:
# 単語分割する関数を定義


def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]


text = '機械学習が好きです。'
print(tokenizer_janome(text))


['機械', '学習', 'が', '好き', 'です', '。']


MeCab

公式サイト

http://taku910.github.io/mecab/

In [19]:
!pip install mecab-python3
!sudo apt install libmecab-dev
!sudo apt install mecab-ipadic-utf8


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmecab-dev is already the newest version (0.996-5).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


モデル（分類タスク用）の実装
本ファイルでは、クラス分類のモデルを実装します。
※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

In [ ]:
import random
import time
import numpy as np
from tqdm import tqdm
import torch 
from torch import nn
import torch.optim as optim
import torchtext

In [ ]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

形態素解析の準備

形態素解析はpipで簡単にインストールできるmecabのラッパーであるfugashiを使います。
下記のように辞書も一緒にpipでインストールできます。

In [21]:
!pip install fugashi
!pip install unidic-lite

     |████████████████████████████████| 490 kB 5.3 MB/s 
     |████████████████████████████████| 47.4 MB 45 kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=d949f2c668c317db4a5ca431c86d85f0969718f05efe48340f429a2544f280dd
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite


形態素解析をする関数は以下のように特に前処理などは施さないシンプルなものにしました。
fugashiは以下のように、mecabと同様の使い方ができます。

In [22]:
import fugashi

tagger = fugashi.Tagger("-Owakati")
def make_wakati(text):
    text = tagger.parse(text)
    wakati = text.split(" ")
    wakati = list(filter(("").__ne__, wakati))
    return wakati

# 形態素解析テスト
text = df.sample(n=1)['text'].item()
print(make_wakati(text)[:30])

['中高生', '向け', 'シリコン', 'バレー', '風', 'IT', '学習', '・', 'テクノロジー', 'キャンプ', '「', 'Life', 'is', 'Tech', '！', '」', 'を', '行う', 'ピス', 'チャー', '株式', '会社', 'は', '、', '2012', '年', '8', '月', '22', '日']


livedoorニュースコーパスを学習データ、検証データ、テストデータの3つに分割しています。
CNNの実装を確かめることがメインなので、単語ベクトルも今回はとりあえず学習データからvocabularyを生成して、ランダムなベクトルを扱うことにします。また、文章の最大長を指定するmax_lengthなどは特に指定していません。

In [23]:
livedoor_df = df.copy()

In [24]:
from sklearn.model_selection import train_test_split
from torchtext.legacy import data

# カテゴリーをidに変換します。
categories = livedoor_df['label_index'].unique().tolist()
livedoor_df['category_id'] = livedoor_df['label_index'].map(lambda x: categories.index(x))

# 元データを学習、検証、テストの３つに分割します。
train_val_df, test_df = train_test_split(livedoor_df[['text', 'label_index']], train_size=0.8)
train_df, val_df = train_test_split(train_val_df, train_size=0.75)

print('train size', train_df.shape)
print('validation size', val_df.shape)
print('test size', test_df.shape)
# train size (4425, 2)
# validation size (1475, 2)
# test size (1476, 2)

# torchtext用にtsvファイルで保存します。
train_df.to_csv('train.tsv', sep='\t', index=False, header=None)
val_df.to_csv('val.tsv', sep='\t', index=False, header=None)
test_df.to_csv('test.tsv', sep='\t', index=False, header=None)

TEXT = data.Field(sequential=True, tokenize=make_wakati, lower=False, batch_first=True, pad_token='<pad>')
LABEL = data.Field(sequential=False, use_vocab=False)

train_data, val_data, test_data = data.TabularDataset.splits(
    path="./", train='train.tsv', validation='val.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])

# vocabulary生成
# 学習データだけでvocabを作成します。
TEXT.build_vocab(train_data, min_freq=1)

BATCH_SIZE = 64
train_loader = data.Iterator(train_data, batch_size=BATCH_SIZE, train=True)
val_loader = data.Iterator(val_data, batch_size=BATCH_SIZE, train=False, sort=False)
test_loader = data.Iterator(test_data, batch_size=BATCH_SIZE, train=False, sort=False)

train size (4425, 2)
validation size (1475, 2)
test size (1476, 2)


CNNによるモデルの定義

自然言語処理におけるCNNの実装解説
自然言語処理では文章を行列（単語ベクトルの集まり）として扱うことが多いですが、その行列を（チャネル1の）画像とみなせば、自然言語に対してCNNを適用することができます。

まずは文章の行列を用意します。要素がランダムなミニバッチサイズ2の7××5の行列を用意しています。
自然言語処理で考えると、長さが7, 単語のベクトル次元数が5の文章を2つ用意した、ということになります。

In [25]:
import torch
import torch.nn as nn

mat = torch.rand(2, 7, 5)
print(mat.size())
#torch.Size([2, 7, 5])
print(mat)

torch.Size([2, 7, 5])
tensor([[[0.6377, 0.8431, 0.1819, 0.7613, 0.3365],
         [0.3940, 0.6882, 0.8789, 0.7536, 0.9835],
         [0.3567, 0.3972, 0.5411, 0.1002, 0.2583],
         [0.6788, 0.7589, 0.7138, 0.7372, 0.4541],
         [0.7387, 0.0464, 0.2618, 0.5235, 0.4555],
         [0.5990, 0.3824, 0.2049, 0.5288, 0.9487],
         [0.7147, 0.5813, 0.3866, 0.8922, 0.0254]],

        [[0.1257, 0.2824, 0.0356, 0.4158, 0.7254],
         [0.7697, 0.0820, 0.6035, 0.8872, 0.7706],
         [0.0806, 0.4681, 0.6841, 0.5024, 0.9049],
         [0.4826, 0.0900, 0.0334, 0.7986, 0.8289],
         [0.7052, 0.5996, 0.4312, 0.7273, 0.2915],
         [0.8440, 0.2770, 0.5588, 0.3321, 0.2168],
         [0.9371, 0.1100, 0.0608, 0.0897, 0.3733]]])


この文章を畳み込みます。畳み込みフィルターのサイズは図の通り、4×5とし、ストライドは1とします。このフィルターを2枚畳み込みたいので、アウトプットのチャネルは2を指定すればOK。
自然言語処理でnn.LSTMなどを扱うとき、インプットの形式は（batch_first=Trueを指定した場合）ミニバッチサイズ×文章の長さ×単語ベクトル次元数のテンソルを扱いますが、nn.Conv2dのインプットの形式はミニバッチサイズ×チャネル数×高さ×幅である必要があります。なので、下記のようにmat.unsqueeze(1)をしてミニバッチサイズの次にチャネル1の次元を追加しています。

In [26]:
# 第１引数はインプットのチャネル（今回は1）を指定
# 自然言語処理で畳み込む場合、異なる単語分散表現（word2vecとfasttextみたいな）などを使って、
# 複数チャネルとみなす方法もあるようです。
# 第２引数はアウトプットのチャネル数で、今回は同じフィルターを2枚畳み込みたいので、2を指定
# カーネルサイズは高さ×幅を指定しており、幅は図で説明した通り、単語ベクトルの次元数5を指定
conv = nn.Conv2d(1, 2, kernel_size=(4, 5))

# チャネル数1を挿入
mat = mat.unsqueeze(1)
print(mat.size())
# torch.Size([2, 1, 7, 5]) 
# ↑ミニバッチサイズ×チャネル数×文章の長さ×単語ベクトル次元数

# 畳み込む
feature = conv(mat)
print(feature.size())
# torch.Size([2, 2, 4, 1])
# ↑ミニバッチサイズ×特徴マップの数×（特徴マップの形式4×1）
print(feature)

torch.Size([2, 1, 7, 5])
torch.Size([2, 2, 4, 1])
tensor([[[[-0.0215],
          [-0.0859],
          [ 0.1515],
          [-0.2844]],

         [[-0.6454],
          [-0.2817],
          [-0.4486],
          [-0.5542]]],


        [[[ 0.2394],
          [-0.0084],
          [-0.1830],
          [ 0.2034]],

         [[-0.1989],
          [-0.4538],
          [-0.0732],
          [-0.2634]]]], grad_fn=<MkldnnConvolutionBackward0>)


In [27]:
import torch.nn.functional as F
feature = F.relu(feature)
print(feature.size())
# ↑ミニバッチサイズ×特徴マップの数×（特徴マップの形式4×1）
print(feature)

torch.Size([2, 2, 4, 1])
tensor([[[[0.0000],
          [0.0000],
          [0.1515],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000],
          [0.0000]]],


        [[[0.2394],
          [0.0000],
          [0.0000],
          [0.2034]],

         [[0.0000],
          [0.0000],
          [0.0000],
          [0.0000]]]], grad_fn=<ReluBackward0>)


1-max poolingを行います。ここですることは要は各特徴マップの最大要素を抽出することになります。つまりnn.MaxPool2dを使って、

In [28]:
# nn.MaxPool1dでも良いですが、そのときは上のfeatureに対して
# feauture.unsqueeze(-1)をして最後の次元の1を除去しましょう。
pool = nn.MaxPool2d(kernel_size=(4, 1))
print(pool(feature))

tensor([[[[0.1515]],

         [[0.0000]]],


        [[[0.2394]],

         [[0.0000]]]], grad_fn=<MaxPool2DWithIndicesBackward0>)


としたくなりますが、上のpoolingのカーネルサイズ(上の4のところ)はインプットとなる特徴マップのサイズに依存するんですよね。特徴マップのサイズは元の文章の長さ（最初の行列の行方向）と畳み込みフィルターのサイズに依存するので、上のようにnn.MaxPool2dでpoolingするレイヤーのインスタンスを宣言しちゃうと、poolingのカーネルサイズを可変にできないので、F.max_pool2dを使って以下のようにpoolingする際、インプットとなる特徴マップのサイズを指定するようにします。

In [29]:
# feature.size()[2]で特徴マップの高さを取得しています。
feature = F.max_pool2d(feature, kernel_size=(feature.size()[2], 1))
print(feature.size())
# torch.Size([2, 2, 1, 1])
print(feature)
# viewを使って次元数を整頓します。
feature = feature.view(-1, 2)
print(feature.size())
# torch.Size([2, 2])
print(feature)

torch.Size([2, 2, 1, 1])
tensor([[[[0.1515]],

         [[0.0000]]],


        [[[0.2394]],

         [[0.0000]]]], grad_fn=<MaxPool2DWithIndicesBackward0>)
torch.Size([2, 2])
tensor([[0.1515, 0.0000],
        [0.2394, 0.0000]], grad_fn=<ViewBackward0>)


あとは同様のことを異なる畳み込みフィルターにも適用して、最後に要素を結合して全結合層にぶち込めばOKですね。

ネットワークの定義

In [30]:
class Net(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Net, self).__init__()
        # 単語分散表現はランダムベクトルを使う
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=TEXT.vocab.stoi['<pad>'])
        # 図の黄色い畳み込みフィルター
        self.conv1 = nn.Conv2d(1, 2, kernel_size=(2, embedding_dim))
        # 図の緑色の畳み込みフィルター
        self.conv2 = nn.Conv2d(1, 2, kernel_size=(3, embedding_dim))
        # 図の赤色の畳み込みフィルター
        self.conv3 = nn.Conv2d(1, 2, kernel_size=(4, embedding_dim))

        # 3つ畳み込みの処理でそれぞれ2次元のベクトルが生成されるので、それらを全て結合して6次元のベクトルとなります。
        # livedoorのカテゴリは9つなので、アウトプットサイズは9を指定
        self.linear = nn.Linear(6, 9)

    def forward(self, input_ids):
        # ①文章の行列を取得
        out = self.embeddings(input_ids)
        # チャネル数1を挿入
        out = out.unsqueeze(1)

        # ②畳み込んでreluに通す
        out1 = F.relu(self.conv1(out))
        out2 = F.relu(self.conv2(out))
        out3 = F.relu(self.conv3(out))

        # ③poolingして、各特徴マップの最大要素を取得
        out1 = F.max_pool2d(out1, kernel_size=(out1.size()[2], 1))
        out2 = F.max_pool2d(out2, kernel_size=(out2.size()[2], 1))
        out3 = F.max_pool2d(out3, kernel_size=(out3.size()[2], 1))

        # ④viewして次元を整えてあげる
        out1 = out1.view(-1, 2)
        out2 = out2.view(-1, 2)
        out3 = out3.view(-1, 2)

        # ⑤全部結合して1本のベクトルにする
        out = torch.cat([out1, out2, out3], dim=1)

        # ⑥全結合層で９つのカテゴリー分類できるように変換
        out = self.linear(out)

        return out

学習
あとはこのネットワークでちゃんと学習できるか確かめて精度を確認して終わりです。
学習部分は以下のように実装しました。
学習データ、検証データともに順調に損失は減っていきますが、検証データの損失が最後らへんで増えてしまいます。

In [ ]:
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm 
VOCAB_SIZE = len(TEXT.vocab.stoi)
EMBEDDING_DIM = 200

net = Net(VOCAB_SIZE, EMBEDDING_DIM)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# GPUの設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# ネットワークをGPUへ送る
net.to(device)
train_loss = []
val_loss = []
train_accuracy = []
val_accuracy = []

for epoch in tqdm(range(30)):

    # 学習
    _train_loss = 0.0
    _train_acc = 0.0
    net.train()
    for batch in tqdm(train_loader):
        inputs = batch.Text.to(device)
        y = batch.Label.to(device)
        optimizer.zero_grad()
        out = net(inputs)
        loss = loss_function(out, y)
        _, preds = torch.max(out, 1)
        loss.backward()
        optimizer.step()
        _train_loss += loss.item()
        _train_acc += torch.sum(preds == y).item()
    train_loss.append(_train_loss)
    train_epoch_acc = _train_acc / len(train_loader.dataset)
    train_accuracy.append(train_epoch_acc)

    # 検証
    _val_loss = 0.0
    _val_acc = 0.0
    net.eval()
    with torch.no_grad():
        for batch in val_loader:
            inputs = batch.Text.to(device)
            y = batch.Label.to(device)
            out = net(inputs)
            loss = loss_function(out, y)
            _, preds = torch.max(out, 1)
            _val_loss += loss.item()
            _val_acc += torch.sum(preds == y).item()
    val_loss.append(_val_loss)
    val_epoch_acc = _val_acc / len(val_loader.dataset)
    val_accuracy.append(val_epoch_acc)

    print("epoch", epoch,
          "\ttrain loss", round(_train_loss, 4), "\ttrain accuracy", round(train_epoch_acc, 4),
          "\tval loss", round(_val_loss, 4), "\tval accuracy", round(val_epoch_acc, 4))

cuda:0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/70 [00:00<?, ?it/s]

...

  0%|          | 0/70 [00:00<?, ?it/s]

epoch 15 	train loss 12.7061 	train accuracy 0.9629 	val loss 16.6231 	val accuracy 0.7953


  0%|          | 0/70 [00:00<?, ?it/s]

epoch 16 	train loss 11.2228 	train accuracy 0.969 	val loss 16.7276 	val accuracy 0.7966


  0%|          | 0/70 [00:00<?, ?it/s]

精度確認
最後にテストデータによる精度（Fスコア）を確認しましょう。

In [ ]:
# 精度確認
from sklearn.metrics import classification_report

with torch.no_grad():
    test_loss = 0.0
    net.eval()
    prediction = []
    answer = []
    for batch in test_loader:
        input_ids = batch.Text.to(device)
        y = batch.Label.to(device)
        out = net(input_ids)
        _, preds = torch.max(out, 1)
        prediction += list(preds.cpu().numpy())
        answer += list(y.cpu().numpy())
print(classification_report(prediction, answer, target_names=categories))

おわりに
自然言語処理に対するCNNの適用例ということでCNNによる文章分類の実装を確認しました。
